## BEFORE RUNNING THIS CODE DOWNLOAD this two folder 
    
### base model and save to DsLabMp/mistralai folder
https://drive.google.com/drive/folders/1uOiC6qAffkiVIw806PEQ0tEuaouCGL7i?usp=sharing

### trained base model checkpoints and savee it to DsLabMp/mistral-realestate-qlora
https://drive.google.com/drive/folders/13sXRAZnrvdL8ABWwDkk1VBhiPmxGYJHN?usp=sharing

## training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

from huggingface_hub import login

login("pastehuggingfacetokenhere")  # 🔒 Replace with your actual token (keep it private!)


In [ ]:
!pip install -q bitsandbytes accelerate peft transformers datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, concatenate_datasets

# Load both datasets
qa_ds = load_dataset("json", data_files="synthetic_real_estate_qa.jsonl")["train"]
viz_ds = load_dataset("json", data_files="filtered_corpus.jsonl")["train"]

# Merge them for fine-tuning
full_ds = concatenate_datasets([qa_ds, viz_ds])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

# Prep for QLoRA
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# ✅ Add pad token if missing
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

# 🧠 Tokenize function
def tokenize(example):
    return tokenizer(example["prompt"] + "\n" + example["response"],
                     truncation=True, padding="max_length", max_length=512)

# ✅ Tokenize dataset
tokenized_ds = full_ds.map(tokenize, remove_columns=full_ds.column_names)

# 🧩 Create data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# ⚙️ Training arguments
training_args = TrainingArguments(
    output_dir="./mistral-realestate-qlora",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_dir="./logs",
)

# 🧪 Trainer setup
trainer = Trainer(
    model=model,
    train_dataset=tokenized_ds,
    args=training_args,
    data_collator=data_collator,
)

# 🚀 Start training
trainer.train()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/1176 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pareshgupta2004 (pareshgupta2004-a-p-shah-institute-of-technology-thane) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.000700
20,2.796500
30,2.061900
40,1.515200
50,1.231600
60,1.081900
70,0.951300
80,0.943400
90,0.868600
100,0.825100


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_r

TrainOutput(global_step=441, training_loss=0.8810503258456449, metrics={'train_runtime': 6795.0872, 'train_samples_per_second': 0.519, 'train_steps_per_second': 0.065, 'total_flos': 7.710242685478502e+16, 'train_loss': 0.8810503258456449, 'epoch': 3.0})

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

## testing and api

In [ ]:
!pip install fastapi uvicorn nest-asyncio transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
# RUN ONLY ONCE AFTER TRAINING
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-v0.1"
save_dir = "/content/drive/MyDrive/DsLabMp/mistralai/Mistral-7B-v0.1"

# Optional: install bitsandbytes if not already
# !pip install bitsandbytes accelerate

# Quantization config for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load and save tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_dir)

# Load and save model in 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)
model.save_pretrained(save_dir)


In [ ]:
torch.cuda.is_available()

True

In [ ]:
# %%writefile api.py
# from fastapi import FastAPI
# from pydantic import BaseModel
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
# from peft import PeftModel

# class Query(BaseModel):
#     prompt: str

# app = FastAPI()

# # Paths
# base_model = "/content/drive/MyDrive/DsLabMp/mistralai/Mistral-7B-v0.1"
# adapter_checkpoint = "/content/drive/MyDrive/DsLabMp/mistral-realestate-qlora/checkpoint-441"

# # Define quant config BEFORE using it
# if torch.cuda.is_available():
#     bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.float16
#     )
# else:
#     bnb_config = None  # Not used for CPU fallback

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     device_map="auto",
#     torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
#     quantization_config=bnb_config
# )

# # Resize the base model's embeddings to match the adapter
# model.resize_token_embeddings(32001)

# # Load QLoRA adapter
# model = PeftModel.from_pretrained(model, adapter_checkpoint)

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(base_model)

# @app.post("/chat")
# async def generate_response(query: Query):
#     prompt = query.prompt.strip()
#     inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model.generate(
#           **inputs,
#           max_new_tokens=150,
#           do_sample=False,  # ❌ Disable sampling
#           temperature=0.0   # ✅ Force deterministic output
#     )

#     result = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     response = result.replace(prompt, "").strip()
#     return {"response": response}
%%writefile api.py
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

class Query(BaseModel):
    prompt: str

app = FastAPI()

base_model = "/content/drive/MyDrive/DsLabMp/mistralai/Mistral-7B-v0.1"
adapter_checkpoint = "/content/drive/MyDrive/DsLabMp/mistral-realestate-qlora/checkpoint-441"

# Quantization
if torch.cuda.is_available():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )
else:
    bnb_config = None

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    quantization_config=bnb_config
)

# Resize embeddings if needed (only if tokenizer was updated)
model.resize_token_embeddings(32001)

# Load adapter
model = PeftModel.from_pretrained(model, adapter_checkpoint)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)

@app.post("/chat")
async def generate_response(query: Query):
    prompt = query.prompt.strip()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=False,
            temperature=0.0
        )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = result.replace(prompt, "").strip()
    return {"response": response}


Writing api.py


In [ ]:
import threading
import uvicorn

def run_uvicorn():
    uvicorn.run("api:app", host="0.0.0.0", port=7860)

# Start the server in a background thread
server_thread = threading.Thread(target=run_uvicorn)
server_thread.start()

In [ ]:
!pip install pyngrok  # if not already installed


In [ ]:

from pyngrok import ngrok
!ngrok config add-authtoken pasteyourngrokauthtokenhere  # 🔒 Replace with your actual token (keep it private!)
public_url = ngrok.connect(7860)
print(f"Your public FastAPI endpoint is: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Your public FastAPI endpoint is: NgrokTunnel: "https://4818-34-143-203-215.ngrok-free.app" -> "http://localhost:7860"


## ONCE YOU'RE DONE GENERATING API LINK , REPLACE THE API LINK IN THE CODE OF REALTEASE BOT streamlit file in pages folder

In [ ]:
## ONCE YOU'RE DONE GENERATING API LINK , REPLACE THE API LINK IN THE CODE OF REALTEASE BOT streamlit file in pages folder